In [2]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.0 MB/s eta 0:00:00


In [3]:
# 02_blast_scoring_and_selection.ipynb
# Step-by-step scoring pipeline for generated enzyme sequences using BLAST

import os
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from collections import Counter
import re

# ----------------------------
# Step 1: Preprocess Sequences
# ----------------------------
INPUT_FASTA = "generated_sequences.fasta"
CLEAN_FASTA = "cleaned_sequences.fasta"
os.makedirs("plots", exist_ok=True)

cleaned_records = []
for record in SeqIO.parse(INPUT_FASTA, "fasta"):
    clean_seq = re.sub(r"MASK_START|<\|.*?\|>", "", str(record.seq)).strip()
    record.seq = clean_seq
    cleaned_records.append(record)

SeqIO.write(cleaned_records, CLEAN_FASTA, "fasta")
print(f"✅ Cleaned sequences saved to {CLEAN_FASTA}")

✅ Cleaned sequences saved to cleaned_sequences.fasta


/usr/local/lib/python3.11/dist-packages/Bio/SeqRecord.py:354: BiopythonDeprecationWarning: Using a string as the sequence is deprecated and will raise a TypeError in future. It has been converted to a Seq object.
  warnings.warn(


In [4]:
# ----------------------------------
# Step 2: Stats and Visualizations
# ----------------------------------
lengths = [len(r.seq) for r in cleaned_records]
families = [r.id.split("_")[0] for r in cleaned_records]
amino_acids = Counter("".join(str(r.seq) for r in cleaned_records))

# Plot: Sequence length distribution
plt.hist(lengths, bins=30, color="skyblue")
plt.title("Sequence Length Distribution")
plt.xlabel("Length")
plt.ylabel("Frequency")
plt.savefig("plots/length_distribution.png")
plt.close()

# Plot: Count per enzyme family
pd.Series(families).value_counts().plot(kind="bar", color="lightgreen")
plt.title("Number of Sequences per Family")
plt.xlabel("Family")
plt.ylabel("Count")
plt.savefig("plots/family_counts.png")
plt.close()

# Plot: Amino acid distribution
aa_df = pd.DataFrame.from_dict(amino_acids, orient="index").sort_index()
aa_df.columns = ["Count"]
aa_df.plot(kind="bar", legend=False, color="salmon")
plt.title("Amino Acid Distribution in Generated Sequences")
plt.xlabel("Residue")
plt.ylabel("Count")
plt.savefig("plots/amino_acid_distribution.png")
plt.close()

print("✅ Plots saved to plots/ directory")

✅ Plots saved to plots/ directory


NExt

In [9]:
# Install BLAST+ in Colab
!apt-get install ncbi-blast+ -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  ncbi-data
The following NEW packages will be installed:
  ncbi-blast+ ncbi-data
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 15.8 MB of archives.
After this operation, 71.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ncbi-data all 6.1.20170106+dfsg1-9 [3,519 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ncbi-blast+ amd64 2.12.0+ds-3build1 [12.3 MB]
Fetched 15.8 MB in 1s (14.2 MB/s)
Selecting previously unselected package ncbi-data.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../ncbi-data_6.1.20170106+dfsg1-9_all.deb ...
Unpacking ncbi-data (6.1.20170106+dfsg1-9) ...
Selecting previously unselected package ncbi-blast+.
Preparing to unpack .../ncbi-blast+_2.12.0+ds-3build1_amd64.deb .

In [10]:
# Cell 2: Python imports and path setup
import os
import pandas as pd
from Bio import SeqIO
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

os.makedirs("scores", exist_ok=True)


In [13]:
import requests

# Corrected query using protein_name instead of name
query = '(reviewed:true) AND (family:"GH5" OR family:"GH10" OR family:"GH48" OR family:"PL1" OR family:"PL7" OR family:"CE1" OR protein_name:"xylanase" OR protein_name:"alginate lyase" OR protein_name:"mannanase" OR protein_name:"cellulase" OR protein_name:"beta-glucosidase" OR protein_name:"esterase")'

url = 'https://rest.uniprot.org/uniprotkb/search'

params = {
    'query': query,
    'format': 'fasta',
    'size': '500'
}

print("Sending request to UniProt...")

response = requests.get(url, params=params)

if response.status_code == 200:
    fasta_data = response.text
    with open('proteins.fasta', 'w') as f:
        f.write(fasta_data)
    print(f"Downloaded {len(fasta_data)} characters and saved to proteins.fasta")
else:
    print(f"Failed to retrieve data, status code: {response.status_code}")
    print("Response content:", response.text)


Sending request to UniProt...
Downloaded 309004 characters and saved to proteins.fasta


In [14]:
# Cell 3: Upload enzyme DB (manual action)
from google.colab import files
uploaded = files.upload()  # Upload `enzymes.fasta` here


Saving enzymes.fasta to enzymes (1).fasta


In [15]:
# Cell 4: Create a protein BLAST database from enzymes.fasta
!makeblastdb -in enzymes.fasta -dbtype prot -out enzymes_db
print("✅ Local BLAST database created.")




Building a new DB, current time: 07/26/2025 18:24:33
New DB name:   /content/enzymes_db
New DB title:  enzymes.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 500 sequences in 0.0247669 seconds.


✅ Local BLAST database created.


In [16]:
# Cell 5: Upload generated enzyme sequences
uploaded = files.upload()  # Upload `cleaned_sequences.fasta`


Saving cleaned_sequences.fasta to cleaned_sequences (1).fasta


In [17]:
# Cell 6: Run BLASTP with local DB
!blastp -query cleaned_sequences.fasta -db enzymes_db -out scores/results.xml -outfmt 5
print("✅ Local BLASTP completed and saved to scores/results.xml")


FASTA-Reader: Ignoring invalid residues at position(s): On line 2: 12
FASTA-Reader: Ignoring invalid residues at position(s): On line 7: 36-37
FASTA-Reader: Ignoring invalid residues at position(s): On line 9: 58-59
FASTA-Reader: Ignoring invalid residues at position(s): On line 13: 5-6
FASTA-Reader: Ignoring invalid residues at position(s): On line 15: 18-19
FASTA-Reader: Ignoring invalid residues at position(s): On line 16: 15-16
FASTA-Reader: Ignoring invalid residues at position(s): On line 20: 12
FASTA-Reader: Ignoring invalid residues at position(s): On line 25: 43-44
FASTA-Reader: Ignoring invalid residues at position(s): On line 28: 31-32
FASTA-Reader: Ignoring invalid residues at position(s): On line 36: 5-6
FASTA-Reader: Ignoring invalid residues at position(s): On line 39: 12
FASTA-Reader: Ignoring invalid residues at position(s): On line 40: 33-34
FASTA-Reader: Ignoring invalid residues at position(s): On line 41: 20-21
FASTA-Reader: Ignoring invalid residues at position(s)

In [18]:
# Cell 7: Parse BLAST XML and extract identity, evalue, top hit
blast_records = NCBIXML.parse(open("scores/results.xml"))
records = list(SeqIO.parse("cleaned_sequences.fasta", "fasta"))
results = []

for record, blast_record in zip(records, blast_records):
    try:
        if blast_record.alignments:
            hit = blast_record.alignments[0]
            hsp = hit.hsps[0]
            results.append({
                "family": record.id.split("_")[0],
                "sample_id": record.id,
                "sequence": str(record.seq),
                "identity": (hsp.identities / hsp.align_length) * 100,
                "evalue": hsp.expect,
                "hit_id": hit.hit_id,
                "hit_def": hit.hit_def,
            })
    except Exception as e:
        print(f"❌ Error for {record.id}: {e}")

df = pd.DataFrame(results)
df.to_csv("scores/blast_results.csv", index=False)
print("✅ BLAST results saved to scores/blast_results.csv")


✅ BLAST results saved to scores/blast_results.csv


In [19]:
# Cell 8: Select the top 2 hits per enzyme family
top_df = df.sort_values(by=["family", "identity"], ascending=[True, False])
top2_df = top_df.groupby("family").head(2).reset_index(drop=True)
top2_df.to_csv("scores/top2_summary.csv", index=False)
print("✅ Top 2 sequences per family saved to scores/top2_summary.csv")


✅ Top 2 sequences per family saved to scores/top2_summary.csv


In [20]:
# Cell 9: Export selected top 2 sequences to a FASTA file
top2_records = []

for _, row in top2_df.iterrows():
    rec = SeqRecord(Seq(row["sequence"]), id=row["sample_id"], description=row["hit_def"])
    top2_records.append(rec)

SeqIO.write(top2_records, "top2_sequences.fasta", "fasta")
print("✅ FASTA of top 2 sequences saved as top2_sequences.fasta")


✅ FASTA of top 2 sequences saved as top2_sequences.fasta
